Load the necessary [romero-gateway](https://github.com/ome/rOMERO-gateway) and [EBImage](http://bioconductor.org/packages/release/bioc/html/EBImage.html)EBImage library:

In [ ]:
options(warn=-1)
library(romero.gateway)
library(EBImage)

## Log into the OMERO server

In [ ]:
#user_name = readline('Username: ')
user_name = 'user-18'   # just use user-18 for now
user_password <- getPass::getPass('OMERO password: ')
server <- OMEROServer(host = 'outreach.openmicroscopy.org', username=user_name, password=user_password, port= as.integer(4064))
server <- connect(server)
paste('Successfully logged in as', server@user$getUserName())

## Load and display an image

Load an image from the example dataset 'R-dataset' (these images are part of the original 'CDK5RAP2-C' dataset).
I'd recommend the fourth image 'siControl_N20_Cep215_I_20110411_Mon-1509_0_SIR_PRJ.dv' as it nicely shows the toroid.

In [ ]:
imageId <- 38255 # An image from R-dataset

image <- loadObject(server, "ImageData", imageId)

Load the pixel values of the second channel (the CDK5RAP2-C channel) and display the image

In [ ]:
pixels <- getPixelValues(image, 1, 1, 2)

ebi <- EBImage::Image(data = pixels, colormode = 'Grayscale')
ebi <- normalize(ebi)
EBImage::display(ebi)

## Image segmentation

### Visual

In [ ]:
threshImg <- thresh(ebi, w=15, h=15, offset=0.1)
threshImg <- medianFilter(threshImg, size=3)
threshImg <- fillHull(threshImg)          
threshImg <- bwlabel(threshImg)
EBImage::display(colorLabels(threshImg))

**Exercise:** Play around with the parameters of the thresh function to get a feeling for how the parameters affect the segmentation, or even plug in other methods into the segmentation workflow to improve the result, see [EBImage documentation](https://www.rdocumentation.org/packages/EBImage/versions/4.14.2)

### Compute the features (measurements)

In [ ]:
shapes = computeFeatures.shape(threshImg)
moments = computeFeatures.moment(threshImg)
features <- merge(shapes, moments, by=0, all=TRUE)
features

## Save the results back to OMERO

### ROIs

We can create an Ellipse for each object taking the x,y coordinates (m.cx and m.cy), the radii (this is the m.majoraxis and calculate the 'minor' axis from 'major' axis and the eccentricity (see documentation of the computeFeatures method)) and the angle theta by which the Ellipse might be rotated.

In [ ]:
createROIs <- function(features) {
    rois <- data.frame(x = c(0), y = c(0), rx = c(0), ry = c(0), w = c(0), h = c(0), theta = c(0), text = c('remove'), stringsAsFactors = FALSE)
    for (index in 1:length(features[,1])) {
        x <- features[index,8]
        y <- features[index,9]
        r1 <- features[index,10]
        ecc <- features[index,11]
        r2 <- sqrt(r1^2 * (1 - ecc^2))
        theta <- features[index,12]
        rois <- rbind(rois, c(x, y, r1, r2, NA, NA, -theta, as.character(index)))
    }
    rois <- rois[-1,]
    rownames(rois) <- c()
    return(rois)
}

In [ ]:
rois <- createROIs(features)
rois

Save the ROIs back to OMERO

In [ ]:
addROIs(image, coords = rois)

**Exercise:** Open the image in OMERO.web and compare the ROIs with the segmented image.

### Features

Attach the whole dataframe to the image:

In [ ]:
invisible(attachDataframe(image, features, "ROI features")) # the invisible just suppresses some unneccessary output

The dataframe is now attached to the image as HDF table. You can download and open it with software like HDF Compass, load it into python scripts using 'h5py' library, etc. or simply load it directly as R dataframe again.

Alternatively attach it as CSV file:

In [ ]:
csvFile <- "/tmp/ROI_features.csv"
write.csv(features, file = csvFile)
fileannotation <- attachFile(image, csvFile)

**Exercise**: Find the attached CSV file and open it in Excel

## Automate

Task: Segment each image of a dataset, create an ROI for each of the objects, and summarize the results as dataframe, only taking the features of the largest ROI of an image into account.

Note: We need to be able to specify the channel which is analyzed by its name.

We put the all the pieces together and wrap them up in a function:

In [ ]:
analyzeImage <- function(image, channelName, df, saveROIs = FALSE) {
    # Find the channel index
    chnames <- getChannelNames(image)
    chindex <- match(channelName, chnames, nomatch = 0)
    if (chindex == 0) {
      message (paste("Could not resolve channel name, skipping ", image@dataobject$getId()))
      return(df)
    }
    
    # Load the pixels
    pixels <- getPixelValues(image, 1, 1, chindex)
    ebi <- EBImage::Image(data = pixels, colormode = 'Grayscale')
    ebi <- normalize(ebi)
     
    # this is our segmentation workflow from above
    threshImg <- thresh(ebi, w=15, h=15, offset=0.1)
    threshImg <- medianFilter(threshImg, size=3)
    threshImg <- fillHull(threshImg)          
    threshImg <- bwlabel(threshImg)
    
    # Calculate the features
    shapes = computeFeatures.shape(threshImg)
    moments = computeFeatures.moment(threshImg)
    features <- merge(shapes, moments, by=0, all=TRUE)
    
    if (length(features[,1])>1) {
        # Add the ROIs to the image
        rois <- createROIs(features)
        if (saveROIs)
            addROIs(image, coords = rois)
        
        # Add the interesting properties (area, perimeter and diameter )
        # of the largest ROI together with channel name, image name, image id 
        # and roi index to the dataframe
        features <- features[order(-features[,2]),]
        diameter <- features[1,4]*2
        df <- rbind(df, c(channelName, image@dataobject$getName(), image@dataobject$getId(), features[1,1], features[1,2], features[1,3], diameter))
    }
    return(df)
}

In [ ]:
datasetId <- 3045 # Insert the ID of your R-dataset here!

channelName <- 'CDK5RAP2-C'

dataset <- loadObject(server, "DatasetData", datasetId)

# Keep the channel name, image name, image id, area, perimeter, and diameter of the largest ROIs
result <- data.frame(Channel = c('remove'), ImageName = c('remove'), Image = c(0), ROIIndex = c(0), Area = c(0), Perimeter = c(0), Diameter = c(0), stringsAsFactors = FALSE)

images <- getImages(dataset)
for (image in images) {
    result <- tryCatch({
        analyzeImage(image, channelName, result)
    }, warning = function(war) {
        message(paste("WARNING:  ", image@dataobject$getId(),war))
        return(result)
    }, error = function(err) {
        message(paste("ERROR:  ", image@dataobject$getId() ,err))
        return(result)
    }, finally = {
    })
}

result <- result[-1,]
rownames(result) <- c()

# set the correct datatypes
result$Channel <- as.factor(result$Channel)
result$Area <- as.numeric(result$Area)
result$Perimeter <- as.numeric(result$Perimeter)
result$Diameter <- as.numeric(result$Diameter)

result

**Exercise**: Check the ROIs in OMERO.web

**Exercise**: Find out the pixel size in nanometer and change the values for Area, Perimeter and Diameter so that they show the values in nanometer instead of pixels.

In [ ]:
# apply the pixel size
pxSizeInNM <- 40
result$Area <- result$Area * pxSizeInNM * pxSizeInNM
result$Perimeter <- result$Perimeter * pxSizeInNM
result$Diameter <- result$Diameter * pxSizeInNM

result

## Analyze

### Trainer only - Prepare the table

Run the segmentation over the whole project, taking into account that the dataset name == the channel which has to be analyzed.

**Do not run this!** Instead we're loading the table which was calculated beforehand.

In [ ]:
projectId <- 979

project <- loadObject(server, "ProjectData", projectId)
datasets <- getDatasets(project)

result <- data.frame(Channel = c('remove'), ImageName = c('remove'), Image = c(0), ROIIndex = c(0), Area = c(0), Perimeter = c(0), Diameter = c(0), stringsAsFactors = FALSE)

count <- 0
for (dataset in datasets) {
    dsname <- dataset@dataobject$getName()
    if (startsWith(dsname, "CEP120"))
        dsname <- "CEP120"
    images <- getImages(dataset)
    for (image in images) {
        result <- tryCatch({
            analyzeImage(image, dsname, result, saveROIs = FALSE)
        }, warning = function(war) {
            message(paste("WARNING:  ", image@dataobject$getId(),war))
            return(result)
        }, error = function(err) {
            message(paste("ERROR:  ", image@dataobject$getId() ,err))
            return(result)
        }, finally = {
        })
        count <- count + 1
        message(count)
    }
}

# remove the first row and row names
result <- result[-1,]
rownames(result) <- c()

# set the correct datatypes
result$Channel <- as.factor(result$Channel)
result$Area <- as.numeric(result$Area)
result$Perimeter <- as.numeric(result$Perimeter)
result$Diameter <- as.numeric(result$Diameter)
result$Image <- as.integer(result$Image)

# set the correct pixel size
pxSizeInNM <- 40
result$Area <- result$Area * pxSizeInNM * pxSizeInNM
result$Perimeter <- result$Perimeter * pxSizeInNM
result$Diameter <- result$Diameter * pxSizeInNM

# Rename the Channel column to Dataset (for parade)
colnames(result)[1] <- "Dataset"
result$Image <- as.integer(result$Image)

result

In [ ]:
# attach the dataframe to the project
invisible(attachDataframe(project, result, "Summary from R"))

# Achtung HACK! Change the namespace (for parade)
dataframes <- availableDataframes(project)
faid <- dataframes$AnnotationID[1]
fa <- loadObject(server, "omero.gateway.model.FileAnnotationData", as.integer(faid))
fa@dataobject$setNameSpace("openmicroscopy.org/omero/bulk_annotations")
gateway <- getGateway(server)
ctx <- getContext(server)
dm <- gateway$getFacility(DataManagerFacility$class)
fa <- dm$saveAndReturnObject(ctx, fa@dataobject)

### Students - Load the table

Instead of running the segmentation over the whole idr0021 project yourself, you'll use the summary table of trainer-1. In OMERO.web got to Lab1 / trainer-1 and find out the ID of the idr0021 project.

In [ ]:
projectId <- 979 # This is the ID of trainer-1's idr0021 project  (at the moment that's actually user-18's which
                                                                 # was created in previous trainer-only step)
project <- loadObject(server, "ProjectData", projectId)
dataframes <- availableDataframes(project)
print(dataframes)

In [ ]:
dfID <- dataframes$ID[1]
df <- loadDataframe(project, dfID)
df

In [ ]:
ag <-aggregate(result$Diameter ~ result$Dataset, result, median)
ordered <- factor(result$Dataset, levels=ag[order(ag$`result$Diameter`), 'result$Dataset'])

In [ ]:
#boxplot(Diameter ~ Dataset, data=result)
plot(result$Diameter ~ ordered, ylab='Diameter (nm)', xlab="Protein", cex.axis=0.5)

In [ ]:
disconnect(server)